In [5]:
tests = [
    {
        "W": 165,
        "weights": [23, 31, 29, 44, 53, 38, 63, 85, 89, 82],
        "costs": [92, 57, 49, 68, 60, 43, 67, 84, 87, 72],
        "ans": [1, 1, 1, 1, 0, 1, 0, 0, 0, 0]
    },
    
    {
        "W": 26,
        "weights": [12,7,11,8,9],
        "costs": [24,13,23,15,16],
        "ans": [0, 1, 1, 1, 0]
    },
    
    {
        "W": 190,
        "weights": [56,59,80,64,75,17],
        "costs": [50,50,64,46,50,5],
        "ans": [1,1,0,0,1,0]
    },
    
    {
        "W": 50,
        "weights": [31,10,20,19, 4, 3, 6],
        "costs": [70,20,39,37,7,5,10],
        "ans": [1,0,0,1,0,0,0]
    },
    
    {
        "W": 104,
        "weights": [25, 35,45, 5,25, 3, 2, 2],
        "costs": [350,400,450, 20, 70,  8,  5,  5],
        "ans": [1,0,1,1,1,0,1,1]
    },
    
    {
        "W": 170,
        "weights": [41, 50, 49,59,55,57,60],
        "costs": [442,525,511,593,546,564,617],
        "ans": [0,1,0,1,0,0,1]
    },
    
    {
        "W": 750,
        "weights": [70,73,77,80,82,87,90,94,98,106,110,113,115,118,120],
        "costs": [135,139,149,150,156,163,173,184,192,201,210,214,221,229,240],
        "ans": [1,0,1,0,1,0,1,1,1,0,0,0,0,1,1]
    },
    
    {
        "W": 6404180,
        "weights": [382745,799601,909247,729069,467902,44328,34610,698150,823460,903959,853665,551830,
                    610856,670702,488960,951111,323046,446298,931161,31385,496951,264724,224916,169684],
        "costs": [ 825594,1677009,1676628,1523970,943972,97426,69666,1296457,1679693,1902996,1844992,
                  1049289,1252836,1319836,953277,2067538,675367,853655,1826027,65731,901489,577243,466257,369261],
        "ans": [1,1,0,1,1,1,0,0,0,1,1,0,1,0,0,1,0,0,0,0,0,1,1,1]
    },
    
]

In [9]:
len(tests)

8

# ДИНАМИКА ПО СТОИМОСТИ

In [101]:

def knapsack_dinamic_cost(N, M, weights, costs):
    '''Динамика по ценности'''
    
    COST = sum(costs)
    
    y = [[0 for i in range(COST+1)] for j in range(N+1)]
    for i in range(1, COST+1):
        y[0][i] = M + 1


    for i in range(1,N+1):
        for V in range(COST+1):

            if costs[i-1] > V:
                y[i][V] = y[i-1][V]
            else:
                y[i][V] = min(y[i-1][V], y[i-1][V-costs[i-1]] + weights[i-1])
        
    #Построили матрицу y[i,V]
    #где i -  используем до i-го предмета
    #V - стоимость которуюю набирвем


    IND = COST
    while y[N][IND] == M+1:
        IND-=1
    #IND - максимальная цена, которую можно набрать используя все предметы


    res = IND



    lis =[0 for i in range(N)]
    for i in range(N, -1,-1):
        if y[i-1][IND] != y[i][IND]:
            lis[i-1]=1
            IND-=costs[i-1]



    return res, lis

In [103]:
def check_correctness(func, test):
    cost, res = func(len(test["weights"]), test["W"],
                      test["weights"], test["costs"])
    
    real_cost = 0
    
    for i in range(len(test["weights"])):
        if test["ans"][i] == 1:
            real_cost += test["costs"][i]
            
    if real_cost == cost and res == test["ans"]:
        return "EXACT"
    if real_cost == cost:
        return "CORRECT", res, test["ans"]
    return "FALSE", res, test["ans"]

In [73]:
%%time
check_correctness(knapsack_dinamic_cost, tests[0])

CPU times: user 2.28 ms, sys: 28 µs, total: 2.31 ms
Wall time: 2.3 ms


'EXACT'

In [74]:
%%time
check_correctness(knapsack_dinamic_cost, tests[1])

CPU times: user 682 µs, sys: 9 µs, total: 691 µs
Wall time: 696 µs


'EXACT'

In [75]:
%%time
check_correctness(knapsack_dinamic_cost, tests[2])

CPU times: user 4.01 ms, sys: 0 ns, total: 4.01 ms
Wall time: 3.56 ms


'EXACT'

In [76]:
%%time
check_correctness(knapsack_dinamic_cost, tests[3])

CPU times: user 2.23 ms, sys: 0 ns, total: 2.23 ms
Wall time: 2.12 ms


'EXACT'

In [77]:
%%time
check_correctness(knapsack_dinamic_cost, tests[4])

CPU times: user 15.8 ms, sys: 0 ns, total: 15.8 ms
Wall time: 15.4 ms


'EXACT'

In [78]:
%%time
check_correctness(knapsack_dinamic_cost, tests[5])

CPU times: user 23.8 ms, sys: 0 ns, total: 23.8 ms
Wall time: 23 ms


'EXACT'

In [79]:
%%time
check_correctness(knapsack_dinamic_cost, tests[6])

CPU times: user 41.2 ms, sys: 14 µs, total: 41.2 ms
Wall time: 39.5 ms


'EXACT'

In [80]:
%%time
check_correctness(knapsack_dinamic_cost, tests[7])

CPU times: user 4min 18s, sys: 2.02 s, total: 4min 20s
Wall time: 4min 20s


'EXACT'

In [ ]:
["2.3 ms", "696 µs", "3.56 ms", "2.12 ms", "15.4 ms", "23 ms", "39.5 ms", "4min 20s"]

# ДИНАМИКА ПО ВЕСАМ

In [455]:

def knapsack_dinamic_weight(N, M, weights, costs):
    '''Динамика по весам'''


    c = [0 for i in range(M+1)]
    lists = [[] for i in range(M+1)] 
    c[0] = 0

    for W in range(1, M+1):
        c[W] = c[W-1]
        lists[W] = lists[W-1]

        for i in range(N):
            w = weights[i]
            if W - w >= 0 and  i not in lists[W-w]:
                    if c[W] < costs[i] + c[W-w]:
                        lists[W] = lists[W-w] + [i]
                        c[W] = costs[i] + c[W-w]
    res = [0 for i in range(N)]
    for i in lists[M]:
        res[i] = 1
        
    return c[M], res

In [92]:
%%time
check_correctness(knapsack_dinamic_weight, tests[0])

CPU times: user 274 µs, sys: 3 µs, total: 277 µs
Wall time: 279 µs


'EXACT'

In [93]:
%%time
check_correctness(knapsack_dinamic_weight, tests[1])

CPU times: user 167 µs, sys: 0 ns, total: 167 µs
Wall time: 176 µs


'EXACT'

In [94]:
%%time
check_correctness(knapsack_dinamic_weight, tests[2])

CPU times: user 0 ns, sys: 931 µs, total: 931 µs
Wall time: 940 µs


'EXACT'

In [95]:
%%time
check_correctness(knapsack_dinamic_weight, tests[3])

CPU times: user 84 µs, sys: 1e+03 ns, total: 85 µs
Wall time: 86.8 µs


'EXACT'

In [96]:
%%time
check_correctness(knapsack_dinamic_weight, tests[4])

CPU times: user 190 µs, sys: 2 µs, total: 192 µs
Wall time: 195 µs


'EXACT'

In [97]:
%%time
check_correctness(knapsack_dinamic_weight, tests[5])

CPU times: user 240 µs, sys: 2 µs, total: 242 µs
Wall time: 245 µs


'EXACT'

In [98]:
%%time
check_correctness(knapsack_dinamic_weight, tests[6])

CPU times: user 2.53 ms, sys: 28 µs, total: 2.56 ms
Wall time: 2.56 ms


'EXACT'

In [99]:
%%time
check_correctness(knapsack_dinamic_weight, tests[7])

CPU times: user 40.1 s, sys: 172 ms, total: 40.3 s
Wall time: 40.2 s


'EXACT'

In [ ]:
["279 µs", "176 µs", "940 µs", "86.8 µs", "195 µs", "245 µs", "2.56 ms", "40.2 s"]

# ГЕНЕТИЧЕСКИЙ АЛГОРИТМ

In [311]:
import random as rnd
import numpy as np
rnd.seed(42)

class Knapsack_DNA():
    def __init__(self, P=10_000, epoch=30, prob_mutation=0.005, num_best=1000):
        
        if num_best > P:
            raise ValueError("num_best > P")
            
        self.P = P
        self.epoch = epoch
        self.prob_mutation = prob_mutation
        self.num_best = num_best
        
    
    
    def __mutate(self, DNA):
        for i in range(len(DNA)):
            if rnd.random() < self.prob_mutation:
                DNA[i] ^= 1
        return DNA
    
    
    
    def __crossover(self, DNA1, DNA2):
        child_DNA = DNA1[:]
        child_DNA[::2] = DNA1[::2]
        child_DNA[1::2] = DNA2[1::2]
        return DNA1
    
    
    
    def __loss_func(self, DNA):
        
        mask = np.where(DNA==1)
        weight = self.weights[mask].sum()
        
        cost = -1
        if weight <= self.Weight:
            cost = self.costs[mask].sum()
        
        return cost
    
    
    def __begin_epoch(self):
        errors = np.array(list(map(lambda x: self.__loss_func(x), self.Population)))
        indexes_of_bests = np.argpartition(-errors, self.num_best)
        Best = self.Population[indexes_of_bests[:self.num_best],:]
        
        
        self.Population[:self.num_best,:] = Best[:,:]
        
        for i in range(self.num_best,
                       self.Population.shape[0]):
            
            fir = rnd.randint(0, self.num_best-1)
            sec = rnd.randint(0, self.num_best-1)
            self.Population[i] = self.__crossover(Best[fir,:], Best[sec,:])
            self.__mutate(self.Population[i])
 
            
        
        
    
    
    def fit(self, test):
        self.N = len(test["weights"])
        self.Weight = test["W"]
        self.weights = np.array(test["weights"])
        self.costs = np.array(test["costs"])
        self.ans = np.array(test["ans"])
        
        self.Population = np.array([[rnd.randint(0,1) for i in range(self.N)] for j in range(self.P)])
        for i in range(self.epoch):
            self.__begin_epoch()
            errors = np.array(list(map(lambda x: self.__loss_func(x), self.Population)))
#             print(errors.max())
            
    def predict(self):
        errors = np.array(list(map(lambda x: self.__loss_func(x), self.Population)))
        pos = errors.argmax()
        
        return self.__loss_func(self.Population[pos,:]), self.Population[pos,:]
    
    def score(self):
        cost, res = self.predict()
        
    
        real_cost = self.costs[np.where(self.ans == 1)].sum()
        
        if real_cost == cost and (res == self.ans).all():
            return "EXACT"
        if real_cost == cost:
            return "CORRECT", res, self.ans
        print("Truth = " + str(real_cost))
        print("Obtain = " + str(cost))
        print("Difference = " + str(real_cost - cost))
        return "FALSE"
        
    

In [312]:
mdl = Knapsack_DNA() 

In [313]:
%%time
mdl.fit(tests[0])
mdl.score()

CPU times: user 6.06 s, sys: 11.9 ms, total: 6.07 s
Wall time: 6.06 s


'EXACT'

In [314]:
%%time
mdl.fit(tests[1])
mdl.score()

CPU times: user 5.71 s, sys: 3.89 ms, total: 5.72 s
Wall time: 5.75 s


'EXACT'

In [315]:
%%time
mdl.fit(tests[2])
mdl.score()

CPU times: user 5.8 s, sys: 7.93 ms, total: 5.81 s
Wall time: 5.84 s


'EXACT'

In [316]:
%%time
mdl.fit(tests[3])
mdl.score()

CPU times: user 6.04 s, sys: 0 ns, total: 6.04 s
Wall time: 6.03 s


'EXACT'

In [317]:
%%time
mdl.fit(tests[4])
mdl.score()

CPU times: user 6.09 s, sys: 3.83 ms, total: 6.09 s
Wall time: 6.09 s


'EXACT'

In [318]:
%%time
mdl.fit(tests[5])
mdl.score()

CPU times: user 5.97 s, sys: 0 ns, total: 5.97 s
Wall time: 5.96 s


'EXACT'

In [319]:
%%time
mdl.fit(tests[6])
mdl.score()

Truth = 1458
Obtain = 1456
Difference = 2
CPU times: user 6.14 s, sys: 16.1 ms, total: 6.16 s
Wall time: 6.17 s


'FALSE'

In [320]:
%%time
mdl.fit(tests[7])
mdl.score()

CPU times: user 6.6 s, sys: 3.89 ms, total: 6.6 s
Wall time: 6.59 s


'EXACT'

In [ ]:
["6.06 s", "5.75 s", "5.84 s", "6.03 s", "6.09 s", "5.96 s", "6.17 s", "6.59 s"]

# ЛИНЕЙНОЕ ПРОГРАММИРОВАНИЕ

$x_i \le 1$

$x_i \ge 0 \Leftrightarrow  -x_i \le 0 $

$\sum_{i=0}^{N}$ weights$_i \cdot x_i \le W$

$\sum_{i=0}^{N}$ costst$_i \cdot x_i \rightarrow max$

In [235]:
import cvxopt
import numpy as np

In [216]:
def linear_programming(N, W, weights, costs):
    b = []
    A = []
    
    c = [-i for i in costs]
    for i in range(N):
        b.append(1)
        
    for i in range(N):
        b.append(0)
        
    b.append(W)
    
    for i in range(N):
        string = [0 for i in range(N)]
        string[i] = 1
        A.append(string)
        
    for i in range(N):
        string = [0 for i in range(N)]
        string[i] = -1
        A.append(string)
        
    string = [i for i in weights]
    A.append(string)
    
    
    
    A = cvxopt.matrix(np.array(A).astype(np.float))
    b = cvxopt.matrix(np.array(b).astype(np.float))
    c = cvxopt.matrix(np.array(c).astype(np.float))
    
    
    sol = cvxopt.glpk.ilp(c, A, b, I={i for i in range(N)})
    sol = list(sol[1])

    return np.array(sol)
    

In [217]:
def check_correctness_lp(test):
    res = linear_programming(len(test["weights"]), test["W"],
                  test["weights"], test["costs"])
    cost = np.dot(res.reshape(-1),np.array(test["costs"]))
    real_cost = np.dot(np.array(test["ans"]),np.array(test["costs"]))
    if cost == real_cost and (res.astype(np.int) == np.array(test["ans"])).all():
        return "EXACT"
        
    if cost == real_cost:
        return "CORRECT", res, test["ans"]
        
    
    return "FALSE", res, test["ans"]
    
    

In [219]:
%%time
check_correctness_lp(tests[0])

CPU times: user 577 µs, sys: 179 µs, total: 756 µs
Wall time: 683 µs


'EXACT'

In [220]:
%%time
check_correctness_lp(tests[1])

CPU times: user 2.12 ms, sys: 0 ns, total: 2.12 ms
Wall time: 2.76 ms


'EXACT'

In [221]:
%%time
check_correctness_lp(tests[2])

CPU times: user 1.32 ms, sys: 410 µs, total: 1.73 ms
Wall time: 3.78 ms


'EXACT'

In [222]:
%%time
check_correctness_lp(tests[3])

CPU times: user 2.1 ms, sys: 649 µs, total: 2.75 ms
Wall time: 5.48 ms


'EXACT'

In [223]:
%%time
check_correctness_lp(tests[4])

CPU times: user 1.98 ms, sys: 0 ns, total: 1.98 ms
Wall time: 1.83 ms


'EXACT'

In [224]:
%%time
check_correctness_lp(tests[5])

CPU times: user 0 ns, sys: 967 µs, total: 967 µs
Wall time: 971 µs


'EXACT'

In [225]:
%%time
check_correctness_lp(tests[6])

CPU times: user 2.55 ms, sys: 0 ns, total: 2.55 ms
Wall time: 2.47 ms


'EXACT'

In [226]:
%%time
check_correctness_lp(tests[7])

CPU times: user 2.91 ms, sys: 899 µs, total: 3.81 ms
Wall time: 5.12 ms


'EXACT'

In [ ]:
["683 µs", "2.76 ms", "3.78 ms", "5.48 ms", "1.83 ms","971 µs", "2.47 ms", "5.12 ms"]

# СРАВНЕНИЕ РЕЗУЛЬТАТОВ

In [229]:
import pandas as pd

In [321]:
tab = pd.DataFrame([["2.3 ms", "696 µs", "3.56 ms", "2.12 ms", "15.4 ms", "23 ms", "39.5 ms", "4min 20s"],
                    ["279 µs", "176 µs", "940 µs", "86.8 µs", "195 µs", "245 µs", "2.56 ms", "40.2 s"],
                    ["6.06 s", "5.75 s", "5.84 s", "6.03 s", "6.09 s", "5.96 s", "6.17 s", "6.59 s"],
                    ["683 µs", "2.76 ms", "3.78 ms", "5.48 ms", "1.83 ms","971 µs", "2.47 ms", "5.12 ms"]
                   ],
                   columns=["test1","test2","test3","test4","test5","test6","test7","test8"]
                   , index=["Дин. Стоимость", "Дин. Вес", "Генетический", "Цел.Лин.Прог"])

In [322]:
tab

,test1,test2,test3,test4,test5,test6,test7,test8
Дин. Стоимость,2.3 ms,696 µs,3.56 ms,2.12 ms,15.4 ms,23 ms,39.5 ms,4min 20s
Дин. Вес,279 µs,176 µs,940 µs,86.8 µs,195 µs,245 µs,2.56 ms,40.2 s
Генетический,6.06 s,5.75 s,5.84 s,6.03 s,6.09 s,5.96 s,6.17 s,6.59 s
Цел.Лин.Прог,683 µs,2.76 ms,3.78 ms,5.48 ms,1.83 ms,971 µs,2.47 ms,5.12 ms


In [326]:
for i in range(tab.shape[0]):
    for j in range(tab.shape[1]):
        tab.iloc[i,j] = "True"

In [327]:
tab

,test1,test2,test3,test4,test5,test6,test7,test8
Дин. Стоимость,True,True,True,True,True,True,True,True
Дин. Вес,True,True,True,True,True,True,True,True
Генетический,True,True,True,True,True,True,True,True
Цел.Лин.Прог,True,True,True,True,True,True,True,True
